Step 1: Preprocess the Text Data (Title Column)

Tokenization: Split each article title into individual words.
Vocabulary Creation: Create a vocabulary from all unique words in the titles and assign each word a unique ID.
Padding: Ensure each title has the same length by padding shorter titles or truncating longer ones.

In [1]:
import pandas as pd
import sentencepiece as spm
from sklearn.model_selection import train_test_split
import numpy as np
import torch

In [2]:
# Load the dataset from Parquet file
dataset_path_parquet = "/dtu/blackhole/1e/203774/final_df_new.parquet"
df = pd.read_parquet(dataset_path_parquet, engine='pyarrow')

# Extract titles for tokenization
titles = df['title'].astype(str).tolist()

# Step 1: Tokenize titles using BPE with reduced vocab size
# Save titles to a temporary file to train BPE model
with open("titles.txt", "w", encoding="utf-8") as f:
    for title in titles:
        f.write(title + "\n")





In [3]:
import zipfile
from huggingface_hub import hf_hub_download
from pathlib import Path
from tqdm import tqdm
from typing import Tuple, List
import torch
import tokenizers
import rich

# Load GloVe embeddings
def load_glove_vectors(filename="glove.6B.300d.txt") -> Tuple[List[str], torch.Tensor]:
    """
    Load the GloVe vectors and parse vocabulary and vectors.

    Args:
        filename (str): Name of the GloVe file to load.

    Returns:
        vocabulary (List[str]): List of words in the GloVe vocabulary.
        vectors (torch.Tensor): Tensor of embedding vectors.
    """
    # Download GloVe file from Hugging Face hub
    path = Path(hf_hub_download(repo_id="stanfordnlp/glove", filename="glove.6B.zip"))
    target_file = path.parent / filename

    # Extract GloVe file if not already extracted
    if not target_file.exists():
        with zipfile.ZipFile(path, "r") as zip_ref:
            zip_ref.extractall(path.parent)

        # Ensure target file exists after extraction
        if not target_file.exists():
            print("Available files:")
            for p in path.parent.iterdir():
                print(p)
            raise ValueError(
                f"Target file `{target_file.name}` can't be found. Check if `{filename}` was properly downloaded."
            )

    # Parse vocabulary and vectors
    vocabulary = []
    vectors = []
    with open(target_file, "r", encoding="utf8") as f:
        for line in tqdm(f.readlines(), desc=f"Parsing {target_file.name}..."):
            word, *vector = line.split()
            vocabulary.append(word)
            vectors.append(torch.tensor([float(v) for v in vector], dtype=torch.float32))
    vectors = torch.stack(vectors)
    return vocabulary, vectors



In [4]:

# Prepare data
glove_vocabulary, glove_vectors = load_glove_vectors()
rich.print(f"glove_vocabulary: type={type(glove_vocabulary)}, length={len(glove_vocabulary)}")
rich.print(f"glove_vectors: type={type(glove_vectors)}, shape={glove_vectors.shape}, dtype={glove_vectors.dtype}")

# Add special tokens
special_tokens = ['<|start|>', '<|unknown|>', '<|pad|>']
glove_vocabulary = special_tokens + glove_vocabulary
glove_vectors = torch.cat([torch.randn(len(special_tokens), glove_vectors.shape[1]), glove_vectors])

# Tokenizer for GloVe
glove_tokenizer = tokenizers.Tokenizer(
    tokenizers.models.WordLevel(vocab={v: i for i, v in enumerate(glove_vocabulary)}, unk_token="<|unknown|>")
)
glove_tokenizer.normalizer = tokenizers.normalizers.BertNormalizer(strip_accents=False)
glove_tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.Whitespace()



Parsing glove.6B.300d.txt...: 100%|██████████| 400001/400001 [00:41<00:00, 9661.76it/s]


glove_vocabulary: type=<class 'list'>, length=400001

glove_vectors: type=<class 'torch.Tensor'>, shape=torch.Size([400001, 300]), dtype=torch.float32

In [5]:
# Tokenize titles using GloVe tokenizer
encoded_titles = [glove_tokenizer.encode(title).ids for title in titles]

In [8]:
encoded_titles

[[687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 12233, 1, 649, 37972, 730, 1304],
 [687, 8452, 

In [ ]:
# # Step 1: Calculate the length of each tokenized title
# title_lengths = [len(title) for title in encoded_titles]

# # Step 2: Analyze the distribution of lengths
# mean_length = np.mean(title_lengths)
# median_length = np.median(title_lengths)
# percentile_90 = np.percentile(title_lengths, 90)  # 90th percentile
# percentile_95 = np.percentile(title_lengths, 95)  # 95th percentile

# # Step 3: Choose a max_length based on the distribution (e.g., 90th percentile)
# max_length = int(percentile_90)  # Adjust this based on your requirements

# print(f"Mean Length: {mean_length}")
# print(f"Median Length: {median_length}")
# print(f"90th Percentile: {percentile_90}")
# print(f"95th Percentile: {percentile_95}")
# print(f"Selected Max Length: {max_length}")

# # Step 4: Pad or truncate the titles based on the selected max_length
# padded_titles = torch.zeros((len(encoded_titles), max_length), dtype=torch.long)
# num_truncated = 0

# for i, encoded in enumerate(encoded_titles):
#     length = min(len(encoded), max_length)
#     padded_titles[i, :length] = torch.tensor(encoded[:length])
#     if len(encoded) > max_length:
#         num_truncated += 1

# print(f"Number of truncated titles: {num_truncated} ({num_truncated / len(encoded_titles) * 100:.2f}%)")

# # Step 5: Split the dataset into training and validation sets
# train_titles, val_titles = train_test_split(padded_titles, test_size=0.2, random_state=42)
# print(f"Training set shape: {train_titles.shape}, Validation set shape: {val_titles.shape}")


Mean Length: 8.0
Median Length: 8.0
90th Percentile: 8.0
95th Percentile: 8.0
Selected Max Length: 8
Number of truncated titles: 0 (0.00%)
Training set shape: torch.Size([80, 8]), Validation set shape: torch.Size([20, 8])


In [9]:
# import matplotlib.pyplot as plt
# plt.hist(title_lengths, bins=20, alpha=0.7, color='blue', edgecolor='black')
# plt.axvline(12, color='red', linestyle='dashed', label='Selected Max Length (12)')
# plt.axvline(mean_length, color='green', linestyle='dashed', label='Mean Length')
# plt.axvline(median_length, color='orange', linestyle='dashed', label='Median Length')
# plt.title("Distribution of Tokenized Title Lengths")
# plt.xlabel("Title Length")
# plt.ylabel("Frequency")
# plt.legend()
# plt.show()

# num_truncated = sum(1 for length in title_lengths if length > 12)
# print(f"Number of truncated titles: {num_truncated} ({num_truncated / len(title_lengths) * 100:.2f}%)")



In [ ]:
# # Determine the maximum length dynamically
# max_length = int(np.percentile([len(title) for title in encoded_titles], 90))  # Use 90th percentile

# # Pad or truncate the tokenized titles
# padded_titles = torch.zeros((len(encoded_titles), max_length), dtype=torch.long)
# for i, encoded in enumerate(encoded_titles):
#     length = min(len(encoded), max_length)
#     padded_titles[i, :length] = torch.tensor(encoded[:length])


In [10]:
# Step 1: Prepare the titles (fill NaNs and tokenize)
titles = df['title'].fillna('').tolist()  # fills any missing values with an empty string and converts the column into a list of strings 
encoded_titles = [glove_tokenizer.encode(title).ids for title in titles] # tokenizes each title into a sequence of integers using the glove tokenizer 

# Step 2: Dynamically determine max length
max_length = max(len(encoded) for encoded in encoded_titles)  # finds the length of the longest tokenized title in encoded titles 
print(f"Dynamically determined max length: {max_length}") # dynamically adjusting the max length ensures that the padding truncation step can handle all sequences without manually specifying a dixed length 

# Step 3: Pad or truncate the tokenized titles
padded_titles = torch.zeros((len(encoded_titles), max_length), dtype=torch.long)  # creates a pytorch tensor initialized with zeros , the shape of the tensor is number of titles , max length 
for i, encoded in enumerate(encoded_titles):
    length = min(len(encoded), max_length)
    padded_titles[i, :length] = torch.tensor(encoded[:length])

# Step 4: Split into train and validation sets
train_data, val_data = train_test_split(padded_titles, test_size=0.2, random_state=42) # splits the padded titles into two subsets , train data used for training the model and validation data used for validating the model during training 
# ensures the model is trained on one portion of the data and evaluated on separate unseen portion to prevent overfitting 

# Step 5: Convert to PyTorch tensors
train_data = train_data.clone().detach() 
val_data = val_data.clone().detach()

print("Data preparation complete.")


Dynamically determined max length: 8
Data preparation complete.


## News Encoder Architecture

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Word Embedding Layer
class WordEmbeddings(nn.Module):
    def __init__(self, embedding_matrix):
        super(WordEmbeddings, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(
            embeddings=torch.tensor(embedding_matrix, dtype=torch.float32),
            freeze=True  # Set to False to fine-tune embeddings
        )

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        return embeddings

# Self-Attention Layer
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, embed_dim, num_heads):
        """
        Multi-Head Self-Attention based on the provided formulas.

        Args:
            embed_dim (int): Embedding dimension of the input.
            num_heads (int): Number of self-attention heads.
        """
        super(MultiHeadSelfAttention, self).__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Separate Query and Value projection matrices for each head
        self.query_matrices = nn.Parameter(torch.randn(num_heads, self.head_dim, self.head_dim))
        self.value_matrices = nn.Parameter(torch.randn(num_heads, self.head_dim, self.head_dim))

    def forward(self, inputs):
        """
        Args:
            inputs: Tensor of shape (batch_size, seq_len, embed_dim).

        Returns:
            output: Tensor of shape (batch_size, seq_len, embed_dim).
            attention_weights: Tensor of shape (batch_size, num_heads, seq_len, seq_len).
        """
        batch_size, seq_len, _ = inputs.size()

        # Reshape input for multi-head computation
        inputs = inputs.view(batch_size, seq_len, self.num_heads, self.head_dim)
        inputs = inputs.permute(0, 2, 1, 3)  # Shape: (batch_size, num_heads, seq_len, head_dim)

        attention_outputs = []
        attention_weights_all_heads = []

        for k in range(self.num_heads):
            # Compute attention scores for head k
            Q_k = self.query_matrices[k]  # Shape: (head_dim, head_dim)
            attention_scores = torch.einsum('bqi,qk,bqj->bij', inputs[:, k, :, :], Q_k, inputs[:, k, :, :])
            attention_weights = F.softmax(attention_scores, dim=-1)  # Shape: (batch_size, seq_len, seq_len)

            # Compute weighted sum for head k
            weighted_sum = torch.einsum('bij,bjd->bid', attention_weights, inputs[:, k, :, :])  # (batch_size, seq_len, head_dim)

            # Apply value projection
            V_k = self.value_matrices[k]  # Shape: (head_dim, head_dim)
            weighted_output = torch.einsum('bqd,dk->bqk', weighted_sum, V_k)  # Shape: (batch_size, seq_len, head_dim)

            attention_outputs.append(weighted_output)
            attention_weights_all_heads.append(attention_weights)

        # Concatenate outputs from all heads
        concat_output = torch.cat(attention_outputs, dim=-1)  # Shape: (batch_size, seq_len, embed_dim)

        # Stack attention weights for all heads
        attention_weights_all_heads = torch.stack(attention_weights_all_heads, dim=1)  # Shape: (batch_size, num_heads, seq_len, seq_len)

        return concat_output, attention_weights_all_heads


class AdditiveWordAttention(nn.Module):
    """
    Implements Additive Word Attention for selecting important words
    to learn a more informative representation of news titles.
    """
    def __init__(self, embed_dim, query_dim):
        """
        Additive Word Attention Network.

        Args:
            embed_dim (int): Dimensionality of the input embeddings (h_w).
            query_dim (int): Dimensionality of the query vector (q_w).
        """
        super(AdditiveWordAttention, self).__init__()
        self.embed_dim = embed_dim
        self.query_dim = query_dim

        # Learnable parameters for attention
        self.Vw = nn.Linear(embed_dim, query_dim, bias=True)  # Linear projection matrix
        self.vw = nn.Parameter(torch.randn(query_dim))        # Bias vector vw
        self.qw = nn.Parameter(torch.randn(query_dim))        # Query vector qw

    def forward(self, inputs):
        """
        Compute word-level additive attention.

        Args:
            inputs (torch.Tensor): Tensor of shape (batch_size, seq_len, embed_dim), 
                                   contextual word representations (h_w).

        Returns:
            weighted_sum (torch.Tensor): Tensor of shape (batch_size, embed_dim), 
                                         final news representation as a weighted sum.
            attention_weights (torch.Tensor): Tensor of shape (batch_size, seq_len), 
                                              attention weights for each word.
        """
        # Step 1: Linear projection to transform inputs into attention space
        projection = self.Vw(inputs)  # Shape: (batch_size, seq_len, query_dim)

        # Step 2: Add the bias vector (vw) to the projection
        projection_with_bias = projection + self.vw  # Shape: (batch_size, seq_len, query_dim)

        # Step 3: Apply tanh activation to introduce non-linearity
        scores = torch.tanh(projection_with_bias)  # Shape: (batch_size, seq_len, query_dim)

        # Step 4: Compute raw attention scores via dot product with the query vector (qw)
        attention_scores = torch.matmul(scores, self.qw)  # Shape: (batch_size, seq_len)

        # Step 5: Normalize the attention scores to probabilities using softmax
        attention_weights = F.softmax(attention_scores, dim=1)  # Shape: (batch_size, seq_len)

        # Step 6: Compute the weighted sum of the input embeddings based on attention weights
        weighted_sum = torch.sum(inputs * attention_weights.unsqueeze(-1), dim=1)  # Shape: (batch_size, embed_dim)

        # Return the final weighted representation and the attention weights
        return weighted_sum, attention_weights



In [13]:
class NewsEncoder(nn.Module):
    def __init__(self, embedding_matrix, embed_dim, num_heads, query_dim, dropout=0.2):
        super(NewsEncoder, self).__init__()
        self.word_embeddings = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.self_attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.additive_attention = nn.Sequential(
            nn.Linear(embed_dim, query_dim),
            nn.Tanh(),
            nn.Linear(query_dim, 1),
        )
        self.reconstruction_head = nn.Linear(embed_dim, embedding_matrix.size(1))  # To reconstruct embeddings
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        x = self.word_embeddings(inputs)  # Shape: (batch_size, seq_len, embed_dim)
        x = self.dropout(x)

        attn_output, attn_weights = self.self_attention(x, x, x)
        attn_scores = self.additive_attention(attn_output).squeeze(-1)
        attn_weights = torch.softmax(attn_scores, dim=1)
        news_representation = torch.sum(attn_weights.unsqueeze(-1) * attn_output, dim=1)

        reconstructed = self.reconstruction_head(news_representation)  # Reconstruct embeddings
        return reconstructed, attn_weights


Test 

In [14]:
# Convert GloVe embeddings to PyTorch tensor
embedding_matrix = glove_vectors  # Assuming `glove_vectors` contains your embedding matrix
embed_dim = embedding_matrix.shape[1]

# Initialize NewsEncoder
num_heads = 15
query_vector_dim = 200
dropout = 0.2

news_encoder = NewsEncoder(
    embedding_matrix=embedding_matrix,
    embed_dim=embed_dim,
    num_heads=num_heads,
    query_dim=query_vector_dim,
    dropout=dropout,
)

print("NewsEncoder initialized with GloVe embeddings.")


NewsEncoder initialized with GloVe embeddings.


Step 6: Testing and Validation
Input Data: Pass some titles through the News Encoder.
Output: Verify that the encoder produces a consistent representation (vector) for each title.

In [16]:
# import matplotlib.pyplot as plt
# import seaborn as sns
# import torch
# from torch.utils.data import DataLoader, TensorDataset

# # Assuming train_data is your tokenized and padded dataset
# batch_size = 64  # Batch size
# epochs = 20  # Number of epochs

# # Create DataLoader
# train_loader = DataLoader(TensorDataset(train_data), batch_size=batch_size, shuffle=True)

# # Loss function and optimizer
# criterion = nn.MSELoss()  # Replace this if needed for unsupervised learning
# optimizer = torch.optim.Adam(user_encoder.parameters(), lr=1e-3)

# # Training loop
# for epoch in range(epochs):
#     user_encoder.train()  # Set the model to training mode
#     total_loss = 0

#     for batch_idx, batch in enumerate(train_loader):
#         optimizer.zero_grad()
#         inputs = batch[0].to(torch.long)  # Ensure inputs are LongTensor
#         representations, attention_weights = user_encoder(inputs)

#         # Compute loss (reconstruction, contrastive, etc.)
#         loss = criterion(representations, user_encoder.word_embeddings(inputs).mean(dim=1))  # Example reconstruction task
#         loss.backward()
#         optimizer.step()

#         # Accumulate loss
#         total_loss += loss.item()

#         # Print batch progress
#         print(
#             f"Epoch {epoch + 1}/{epochs} - Batch {batch_idx + 1}/{len(train_loader)}, "
#             f"Loss: {loss.item():.4f}"
#         )

#         # Visualize attention map for the first batch
#         if batch_idx == 0 and epoch == 0:
#             # Reshape the attention weights for visualization
#             reshaped_weights = attention_weights[0].unsqueeze(0)  # Shape: (1, 20)

#             # Plot the heatmap
#             plt.figure(figsize=(10, 6))
#             sns.heatmap(reshaped_weights.cpu().detach().numpy(), cmap="viridis", annot=True)
#             plt.title("Attention Weights for First Sample")
#             plt.xlabel("Sequence Length")
#             plt.ylabel("Attention Heads (Single Row)")
#             plt.show()


#     # Print epoch loss
#     print(f"Epoch {epoch + 1}/{epochs}, Average Loss: {total_loss / len(train_loader):.4f}")


## More tests 

In [17]:
import torch

# Extract the first 5 rows of the train_titles tensor
sample_titles = train_data[:5]  # Slice the first 5 rows

# Pass the sample titles through the News Encoder
news_representations, attention_weights = news_encoder(sample_titles)

# Display the output shapes
print("Sample Titles Shape:", sample_titles.shape)  # Expected shape: (5, max_length)
print("News Representations Shape:", news_representations.shape)  # Expected shape: (5, embed_dim)
print("Attention Weights Shape:", attention_weights.shape)  # Expected shape: (5, max_length)


Sample Titles Shape: torch.Size([5, 8])
News Representations Shape: torch.Size([5, 300])
Attention Weights Shape: torch.Size([5, 8])


In [18]:
import pandas as pd

# Convert attention weights to a DataFrame for readability
attention_df = pd.DataFrame(attention_weights.detach().numpy())

# Add column headers to identify word positions
attention_df.columns = [f"Word {i+1}" for i in range(max_length)]

# Display the DataFrame
print("Attention Weights (as Table):")
print(attention_df)


Attention Weights (as Table):
     Word 1    Word 2    Word 3    Word 4    Word 5    Word 6    Word 7  \
0  0.125864  0.125133  0.124144  0.122410  0.125003  0.125730  0.126432   
1  0.125766  0.125920  0.125110  0.121812  0.125160  0.125389  0.125510   
2  0.125309  0.125476  0.124400  0.122794  0.124897  0.125673  0.125564   
3  0.125814  0.125859  0.124665  0.120811  0.125135  0.125010  0.126634   
4  0.125539  0.125196  0.125092  0.122263  0.124958  0.125549  0.125859   

     Word 8  
0  0.125285  
1  0.125334  
2  0.125887  
3  0.126071  
4  0.125544  


Check Validity

In [19]:
import torch


# Sum the attention weights across the sequence dimension (axis=1)
attention_sum = torch.sum(attention_weights, dim=1)  # Shape: (batch_size, 1)

# Print the result to verify it sums to ~1 for each sequence in the batch
print("Attention Weights Sum (per sequence):\n", attention_sum.detach().numpy())


Attention Weights Sum (per sequence):
 [1.0000001  1.0000001  0.99999994 0.9999999  1.        ]


In [20]:
import torch
from torch.nn.functional import cosine_similarity

# Select two representations to compare
vector1 = news_representations[3]
vector2 = news_representations[4]

# Compute cosine similarity using PyTorch
similarity = cosine_similarity(vector1.unsqueeze(0), vector2.unsqueeze(0)).item()
print("Cosine similarity between Title 1 and Title 2 representations:", similarity)


Cosine similarity between Title 1 and Title 2 representations: 0.8735371828079224


# User Encoder

Step 1: Input Data Preparation
Input: A batch of news representations (output from the News Encoder) for all news articles browsed by users.
The input shape will be (batch_size, num_articles, embed_dim), where:
num_articles: Number of news articles browsed by the user.
embed_dim: The dimension of each news representation (from the News Encoder).

Step 2: Multi-Head Self-Attention Layer
Apply multi-head self-attention to capture relationships between the user's browsed news articles.
Process:
Compute attention scores between each pair of articles.
Learn contextualized news representations by aggregating information across related articles.

Step 3: Additive News Attention Layer
Assign importance weights to each news article using an additive attention mechanism.
Compute a weighted sum of the news representations based on these attention weights.
Output: A single dense vector for each user, representing their preferences.

Step 4: Define the User Encoder Class
Integrate the multi-head self-attention and additive attention layers into a single class.
Handle the processing of input data (news representations) and return the user representation

### User Architecture

In [21]:

class MultiHeadAdditiveAttention(nn.Module):
    """
    Implements multi-head additive attention for user encoding,
    strictly following the provided formulas.
    """
    def __init__(self, embed_dim, num_heads):
        """
        Args:
        - embed_dim: Dimensionality of the input embeddings (news representations).
        - num_heads: Number of attention heads.
        """
        super(MultiHeadAdditiveAttention, self).__init__()
        assert embed_dim % num_heads == 0, "embed_dim must be divisible by num_heads"

        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads

        # Parameters for each head
        self.Q_n = nn.ParameterList([nn.Parameter(torch.randn(self.head_dim, self.head_dim)) for _ in range(num_heads)])
        self.V_n = nn.ParameterList([nn.Parameter(torch.randn(self.head_dim, self.head_dim)) for _ in range(num_heads)])

    def forward(self, news_embeddings):
        """
        Args:
        - news_embeddings: Tensor of shape (batch_size, num_news, embed_dim),
                           representing the news representations.

        Returns:
        - enhanced_news_embeddings: Tensor of shape (batch_size, num_news, embed_dim),
                                    enhanced news representations.
        - attention_weights: List of tensors of shape (batch_size, num_news, num_news) per head,
                             attention weights for each head.
        """
        batch_size, num_news, embed_dim = news_embeddings.size()

        # Split embeddings for each head
        news_per_head = news_embeddings.view(batch_size, num_news, self.num_heads, self.head_dim).transpose(1, 2)

        all_head_outputs = []
        all_attention_weights = []

        for h in range(self.num_heads):
            Q_n = self.Q_n[h]
            scores = torch.einsum('bnd,dk,bmk->bnm', news_per_head[:, h, :, :], Q_n, news_per_head[:, h, :, :])
            attention_weights = F.softmax(scores, dim=-1)
            V_n = self.V_n[h]
            head_output = torch.einsum('bnm,bmd,dk->bnd', attention_weights, news_per_head[:, h, :, :], V_n)

            all_head_outputs.append(head_output)
            all_attention_weights.append(attention_weights)

        concat_output = torch.cat(all_head_outputs, dim=-1)
        return concat_output, all_attention_weights


### Test

In [22]:
# Test configuration
batch_size = 4
num_news = 10
embed_dim = 300
num_heads = 10

# Simulated input
news_embeddings = torch.randn(batch_size, num_news, embed_dim)

# Initialize and test the module
attention_layer = MultiHeadAdditiveAttention(embed_dim=embed_dim, num_heads=num_heads)
enhanced_news_embeddings, attention_weights = attention_layer(news_embeddings)

# Validate outputs
print("Enhanced News Embeddings Shape:", enhanced_news_embeddings.shape)  # Expected: (batch_size, num_news, embed_dim)
print("Attention Weights Shape (per head):", attention_weights[0].shape)  # Expected: (batch_size, num_news, num_news)


Enhanced News Embeddings Shape: torch.Size([4, 10, 300])
Attention Weights Shape (per head): torch.Size([4, 10, 10])


In [23]:

class UserAdditiveAttention(nn.Module):
    """
    Implements additive attention for user encoding based on the provided formulas.
    """
    def __init__(self, embed_dim):
        """
        Args:
        - embed_dim: Dimensionality of the input embeddings (news representations).
        """
        super(UserAdditiveAttention, self).__init__()
        self.V_n = nn.Linear(embed_dim, embed_dim, bias=False)
        self.v_n = nn.Parameter(torch.zeros(embed_dim))
        self.q_n = nn.Parameter(torch.randn(embed_dim))

    def forward(self, news_embeddings):
        """
        Args:
        - news_embeddings: Tensor of shape (batch_size, num_news, embed_dim),
                           representing the news representations.

        Returns:
        - user_representation: Tensor of shape (batch_size, embed_dim),
                               the final user representation.
        - attention_weights: Tensor of shape (batch_size, num_news),
                             attention weights for the news articles.
        """
        transformed_news = self.V_n(news_embeddings)
        scores = torch.tanh(transformed_news + self.v_n)
        scores = torch.einsum('bnd,d->bn', scores, self.q_n)
        attention_weights = F.softmax(scores, dim=1)
        user_representation = torch.einsum('bn,bnd->bd', attention_weights, news_embeddings)
        return user_representation, attention_weights

### Test

In [24]:
# Parameters
batch_size = 16
num_news = 10  # Number of news articles browsed by each user
embed_dim = 300

# Dummy input
news_embeddings = torch.rand(batch_size, num_news, embed_dim)  # Random news embeddings

# Initialize the UserAdditiveAttention layer
user_attention_layer = UserAdditiveAttention(embed_dim=embed_dim)

# Forward pass
user_representation, attention_weights = user_attention_layer(news_embeddings)

# Output shapes
print("User representation shape:", user_representation.shape)  # Expected: (16, 300)
print("Attention weights shape:", attention_weights.shape)  # Expected: (16, 10)


User representation shape: torch.Size([16, 300])
Attention weights shape: torch.Size([16, 10])


In [25]:
class UserEncoder(nn.Module):
    """
    Combines MultiHeadAdditiveAttention and UserAdditiveAttention
    to encode user representations based on news embeddings.
    """
    def __init__(self, embed_dim, num_heads):
        """
        Args:
        - embed_dim: Dimensionality of the input embeddings (news representations).
        - num_heads: Number of attention heads in the MultiHeadAdditiveAttention layer.
        """
        super(UserEncoder, self).__init__()
        self.multi_head_attention = MultiHeadAdditiveAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.user_attention = UserAdditiveAttention(embed_dim=embed_dim)

    def forward(self, news_embeddings):
        """
        Args:
        - news_embeddings: Tensor of shape (batch_size, num_news, embed_dim),
                           representing the news representations.

        Returns:
        - user_representation: Tensor of shape (batch_size, embed_dim),
                               the final user representation.
        - attention_weights: Dictionary containing:
            - 'multi_head_attention': List of tensors of shape (batch_size, num_news, num_news) per head,
                                      attention weights for each head from the MultiHeadAdditiveAttention layer.
            - 'user_attention': Tensor of shape (batch_size, num_news),
                                attention weights for the news articles from the UserAdditiveAttention layer.
        """
        enhanced_news_embeddings, multi_head_attention_weights = self.multi_head_attention(news_embeddings)
        user_representation, user_attention_weights = self.user_attention(enhanced_news_embeddings)

        attention_weights = {
            'multi_head_attention': multi_head_attention_weights,
            'user_attention': user_attention_weights
        }
        return user_representation, attention_weights

### Test

In [26]:
import torch

# Define parameters
embed_dim = 300    # Embedding dimension
num_heads = 10     # Number of attention heads
batch_size = 4     # Number of users in the batch
num_articles = 5   # Number of articles browsed by each user

# Simulate input data (news representations from News Encoder)
user_input = torch.randn(batch_size, num_articles, embed_dim)  # Random tensor for simulation

# Initialize the UserEncoder
user_encoder = UserEncoder(embed_dim=embed_dim, num_heads=num_heads)


### Test

In [27]:
# Parameters
batch_size = 16
num_news = 10  # Number of news articles browsed by each user
embed_dim = 300
num_heads = 10

# Dummy input
news_embeddings = torch.rand(batch_size, num_news, embed_dim)  # Random news embeddings

# Initialize the UserEncoder
user_encoder = UserEncoder(embed_dim=embed_dim, num_heads=num_heads)

# Forward pass
user_representation, attention_weights = user_encoder(news_embeddings)

# Output shapes
print("User representation shape:", user_representation.shape)  # Expected: (16, 300)
print("Multi-head attention weights shape (head 0):", attention_weights['multi_head_attention'][0].shape)  # Expected: (16, 10, 10)
print("User attention weights shape:", attention_weights['user_attention'].shape)  # Expected: (16, 10)


User representation shape: torch.Size([16, 300])
Multi-head attention weights shape (head 0): torch.Size([16, 10, 10])
User attention weights shape: torch.Size([16, 10])


# Tests

Test the Execution of every aspect of the Encoder

In [28]:
def test_user_encoder_initialization(user_encoder):
    assert user_encoder is not None, "UserEncoder initialization failed."
    print("UserEncoder initialization test passed.")

def test_user_encoder_forward(user_encoder, news_embeddings):
    user_representation, attention_weights = user_encoder(news_embeddings)
    assert user_representation.shape == (16, 64), "User representation shape mismatch!"
    assert 'multi_head_attention' in attention_weights, "Missing multi_head_attention key!"
    assert 'user_attention' in attention_weights, "Missing user_attention key!"
    print("UserEncoder forward pass test passed.")

def test_multi_head_attention(multi_head_attention, news_embeddings):
    enhanced_news_embeddings, attention_weights = multi_head_attention(news_embeddings)
    assert enhanced_news_embeddings.shape == (16, 10, 64), "Multi-head attention output shape mismatch!"
    print("MultiHeadAdditiveAttention test passed.")

def test_user_additive_attention(user_attention, news_embeddings):
    user_representation, attention_weights = user_attention(news_embeddings)
    assert user_representation.shape == (16, 64), "User representation shape mismatch!"
    assert attention_weights.shape == (16, 10), "Attention weights shape mismatch!"
    print("UserAdditiveAttention test passed.")




In [29]:
def main():
    # Initialize test inputs
    batch_size = 16
    num_news = 10
    embed_dim = 64
    num_heads = 4

    # Create random news embeddings
    news_embeddings = torch.randn(batch_size, num_news, embed_dim)
    print("News embeddings initialized:", news_embeddings.shape)

    # Initialize UserEncoder
    user_encoder = UserEncoder(embed_dim=embed_dim, num_heads=num_heads)
    print("UserEncoder initialized.")

    # Test: Initialization
    test_user_encoder_initialization(user_encoder)

    # Test: Forward pass
    test_user_encoder_forward(user_encoder, news_embeddings)

    # Test: MultiHeadAdditiveAttention
    multi_head_attention = MultiHeadAdditiveAttention(embed_dim=embed_dim, num_heads=num_heads)
    test_multi_head_attention(multi_head_attention, news_embeddings)

    # Test: UserAdditiveAttention
    user_attention = UserAdditiveAttention(embed_dim=embed_dim)
    test_user_additive_attention(user_attention, news_embeddings)

if __name__ == "__main__":
    main()

News embeddings initialized: torch.Size([16, 10, 64])
UserEncoder initialized.
UserEncoder initialization test passed.
UserEncoder forward pass test passed.
MultiHeadAdditiveAttention test passed.
UserAdditiveAttention test passed.


Everything seems to be executed

In [30]:
# Debugging Script

import torch

# Define Parameters
batch_size = 4    # Number of users in the batch
num_articles = 5  # Number of articles browsed per user
embed_dim = 300   # Embedding dimension (output from News Encoder)
num_heads = 10    # Number of attention heads

# Generate Input Data
user_input = torch.randn(batch_size, num_articles, embed_dim)
print("User Input Shape:", user_input.shape)

# Initialize the User Encoder
try:
    user_encoder = UserEncoder(embed_dim=embed_dim, num_heads=num_heads)
    print("UserEncoder initialized successfully.")
except Exception as e:
    print(f"Error during UserEncoder initialization: {e}")

# Forward Pass
try:
    user_representation, attention_weights = user_encoder(user_input)
    print("Forward pass completed successfully.")
except Exception as e:
    print(f"Error during forward pass: {e}")

# Validate Output Shapes
try:
    print("User Representation Shape:", user_representation.shape)  # Expected: (batch_size, embed_dim)
    print("Attention Weights Shape:", attention_weights.shape)      # Expected: (batch_size, num_articles)
except Exception as e:
    print(f"Error validating output shapes: {e}")

# Inspect Attention Weights for the First User
try:
    print("\nAttention Weights for User 1:\n", attention_weights[0].detach().numpy())  # Shape: (num_articles)
except Exception as e:
    print(f"Error inspecting attention weights: {e}")


User Input Shape: torch.Size([4, 5, 300])
UserEncoder initialized successfully.
Forward pass completed successfully.
User Representation Shape: torch.Size([4, 300])
Error validating output shapes: 'dict' object has no attribute 'shape'
Error inspecting attention weights: 0
